In [ ]:
!pip install -e $PWD/awscosts

In [ ]:
# dataframe-related imports
import wikimedia_scraper as ws
from datetime import datetime
import pandas as pd
import numpy as np

import webish_simulator


# plotting-related imports
import matplotlib.pylab as plt
%matplotlib inline

# Get data from wikipedia source

In [ ]:
project = 'en'

start_date = datetime(2016, 11,  1)
end_date   = datetime(2017, 1, 31)

ws.output_notebook()

traffic_generator = ws.get_traffic_generator(start_date, end_date, projects=(project,))
df = pd.DataFrame(list(traffic_generator))

df.head()

In [ ]:
df = df.set_index(pd.DatetimeIndex(df['date']))
df = df.drop(['date'], axis=1)
df = df.loc[df['project']==project]

df.head()

## Simulate

In [ ]:
month_df = webish_simulator.simulate(df, monthly_scale_factor=30000000)

month_df.head()

## Calculate costs

In [ ]:
month_df = webish_simulator.get_cost(month_df, MB_per_request=512, ms_per_req=200, max_reqs_per_second=1000)

month_df.tail()

# Plot graphs

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

layout = go.Layout(
    title='WW Traffic Monthly Cost',
    legend=dict(orientation="h"),
    xaxis=dict(
        title='Day of month',
        autorange=True
    ),
    yaxis=dict(
        title='Cost ($)'
    )
)

data = []

lambda_trace = go.Scatter(
    x=month_df.index,
    y=month_df['lambda_sum'],
    name='Lambda'
)

data.append(lambda_trace)

ec2_trace = go.Scatter(
    x=month_df.index,
    y=month_df['ec2_sum'],
    name='EC2'
)

data.append(ec2_trace)

fig = go.Figure(data=data, layout=layout)
iplot(fig)